Materialize query history system table for best performance, and so that primary key optimizations can be applied.

See [query optimization using primary key constraints](https://docs.databricks.com/aws/en/sql/user/queries/query-optimization-constraints) for more details.

In [0]:
use catalog identifier(:catalog);
use schema identifier(:schema);

In [0]:
create or replace table fct_query_history(
  calendar_key int comment 'FK for calendar dimension (dim_calendar).',
  compute_key string comment 'FK for compute dimension (dim_compute).',
  workspace_key string comment 'FK for workspace dimension (dim_workspace).',
  statement_id string comment 'The ID that uniquely identifies the execution of the statement. You can use this ID to find the statement execution in the Query History UI.',
  executed_by string comment 'The email address or username of the user who ran the statement.',
  session_id string comment 'The Spark session ID.',
  execution_status string comment 'The statement termination state. Possible values are:
finished: execution was successful
failed: execution failed with the reason for failure described in the accompanying error message
canceled: execution was canceled',
  executed_by_user_id string comment 'The ID of the user who ran the statement.',
  statement_text string comment 'Text of the SQL statement. If you have configured customer-managed keys, statement_text is empty.',
  statement_type string comment 'The statement type. For example: alter, copy, and`insert`.',
  error_message string comment 'Message describing the error condition. If you have configured customer-managed keys, error_message is empty.',
  client_application string comment 'Client application that ran the statement. For example: Databricks SQL, Tableau, and Power BI. Nulls and blanks are labled as Unknown',
  client_driver string comment 'The connector used to connect to Databricks to run the statement. For example: Databricks SQL Driver for Go, Databricks ODBC Driver, Databricks JDBC Driver.',
  total_duration_ms bigint comment 'Total execution time of the statement in milliseconds ( excluding result fetch time ).',
  waiting_for_compute_duration_ms bigint comment 'Time spent waiting for compute resources to be provisioned in milliseconds.',
  waiting_at_capacity_duration_ms bigint comment 'Time spent waiting in queue for available compute capacity in milliseconds.',
  execution_duration_ms bigint comment 'Time spent executing the statement in milliseconds.',
  compilation_duration_ms bigint comment 'Time spent loading metadata and optimizing the statement in milliseconds.',
  total_task_duration_ms bigint comment 'The sum of all task durations in milliseconds. This time represents the combined time it took to run the query across all cores of all nodes. It can be significantly longer than the wall-clock duration if multiple tasks are executed in parallel. It can be shorter than the wall-clock duration if tasks wait for available nodes.',
  result_fetch_duration_ms bigint comment 'Time spent, in milliseconds, fetching the statement results after the execution finished.',
  start_time timestamp comment 'The time when Databricks received the request. Timezone information is recorded at the end of the value with +00:00 representing UTC.',
  end_time timestamp comment 'The time the statement execution ended, excluding result fetch time. Timezone information is recorded at the end of the value with +00:00 representing UTC.',
  update_time timestamp comment 'The time the statement last received a progress update. Timezone information is recorded at the end of the value with +00:00 representing UTC.',
  read_partitions bigint comment 'The number of partitions read after pruning.',
  pruned_files bigint comment 'The number of pruned files.',
  read_files bigint comment 'The number of files read after pruning.',
  read_rows bigint comment 'Total number of rows read by the statement.',
  produced_rows bigint comment 'Total number of rows returned by the statement.',
  read_bytes bigint comment 'Total size of data read by the statement in bytes.',
  read_io_cache_percent tinyint comment 'The percentage of bytes of persistent data read from the IO cache.',
  from_result_cache boolean comment 'true indicates that the statement result was fetched from the cache.',
  spilled_local_bytes bigint comment 'Size of data, in bytes, temporarily written to disk while executing the statement.',
  written_bytes bigint comment 'The size in bytes of persistent data written to cloud object storage.',
  shuffle_read_bytes bigint comment 'The total amount of data in bytes sent over the network.',
  executed_as_user_id string comment 'The ID of the user or service principal whose privilege was used to run the statement.',
  executed_as string comment 'The name of the user or service principal whose privilege was used to run the statement.',
  constraint fk_fct_query_history_dim_calendar
    foreign key (calendar_key) references dim_calendar (calendar_key),
  constraint fk_fct_query_history_dim_compute
    foreign key (compute_key) references dim_compute (compute_key),
  constraint fk_fct_query_history_dim_workspace
    foreign key (workspace_key) references dim_workspace (workspace_key)
)
cluster by (calendar_key, compute_key, workspace_key);

In [0]:
insert overwrite fct_query_history
select
cast(start_time as date) as start_date,
year(start_date) * 10000 + month(start_date) * 100 + day(start_date) as calendar_key,
case 
  when compute.type = 'WAREHOUSE' then compute.warehouse_id
  when compute.type = 'SERVERLESS_COMPUTE' THEN 'serverless'
  else '-1'
end as compute_key,
workspace_id as workspace_key,
statement_id,
executed_by,
session_id,
execution_status,
executed_by_user_id,
statement_text,
statement_type,
error_message,
case
  when client_application is null or client_application in ('', 'unknown') then 'Unknown'
  else client_application
end as client_application,
client_driver,
total_duration_ms,
waiting_for_compute_duration_ms,
waiting_at_capacity_duration_ms,
execution_duration_ms,
compilation_duration_ms,
total_task_duration_ms,
result_fetch_duration_ms,
start_time,
end_time,
update_time,
read_partitions,
pruned_files,
read_files,
read_rows,
produced_rows,
read_bytes,
read_io_cache_percent,
from_result_cache,
spilled_local_bytes,
written_bytes,
shuffle_read_bytes,
executed_as_user_id,
executed_as
from system.query.history
where start_time >= current_date() - interval 6 months
|> drop start_date;

In [0]:
optimize fct_query_history

In [0]:
analyze table fct_query_history compute statistics for all columns;

In [0]:
vacuum fct_query_history